In [17]:
import cv2 
import os

In [18]:
path = 'images'
raw_vid_path = 'raw_video'
moods = os.listdir(path)

def video_detect(video_files, save_folder):
    print("Found",len(video_files))
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    j = 1
    
    for file in video_files:
        print("Extracting",file)
        video = cv2.VideoCapture(file)
        if not video.isOpened():
            print(f"Error: Could not open video file {file}")
            continue
        i = 0
        while True:
            i += 1
            ret, frame = video.read()
            if i!=2: continue
            i = 0
            if not ret:
                break
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            faces = face_cascade.detectMultiScale(gray, 1.5, 5)

            for (x, y, w, h) in faces:
                roi_gray = gray[y:y + h, x:x + w]
                roi = cv2.resize(roi_gray, (224,224), interpolation=cv2.INTER_LINEAR)
                cv2.imwrite(os.path.join(save_folder, f'image_{j}.jpg'), roi)
                j += 1
        video.release()

for mood in moods:
  print(mood)
  files = os.listdir(f'{raw_vid_path}/{mood}')
  video_files = [os.path.join(raw_vid_path,mood,file) for file in files if file.endswith(('.mp4', '.avi', '.mov'))]
  video_detect(video_files,f'{path}/{mood}')

Angry
Found 1
Extracting raw_video\Angry\Angry.mp4
Confused
Found 1
Extracting raw_video\Confused\Confused.mp4
Happy
Found 1
Extracting raw_video\Happy\Happy.mp4
Sad
Found 1
Extracting raw_video\Sad\Sad.mp4


# Image Generation


In [19]:
from keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator(rescale=1./255,
                         validation_split= 0.2,
                         )

train_gen = image_gen.flow_from_directory('images', batch_size=32, target_size=(224,224), class_mode='categorical', subset='training')
test_gen = image_gen.flow_from_directory('images', batch_size=32, target_size=(224,224), class_mode='categorical', subset='validation')

Found 516 images belonging to 4 classes.


Found 127 images belonging to 4 classes.


#  CNN

In [20]:
from keras.models import Sequential
from keras.layers import MaxPooling2D, Dense, Dropout, Flatten, Conv2D

In [21]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(units=4, activation='softmax')  # Specify the number of output units (4 for 4 classes)
])

In [22]:
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])

history = model.fit(train_gen, verbose=1, epochs=25, validation_data=test_gen)


Epoch 1/25
17/17 [==============================] - 10s 549ms/step - loss: 1.4980 - accuracy: 0.6395 - val_loss: 0.0182 - val_accuracy: 1.0000
Epoch 2/25
17/17 [==============================] - 8s 473ms/step - loss: 0.0122 - accuracy: 1.0000 - val_loss: 1.3272e-06 - val_accuracy: 1.0000
Epoch 3/25
17/17 [==============================] - 8s 451ms/step - loss: 9.4455e-04 - accuracy: 1.0000 - val_loss: 3.3134e-07 - val_accuracy: 1.0000
Epoch 4/25
17/17 [==============================] - 8s 452ms/step - loss: 4.0170e-04 - accuracy: 1.0000 - val_loss: 1.7320e-05 - val_accuracy: 1.0000
Epoch 5/25
17/17 [==============================] - 8s 466ms/step - loss: 6.8290e-05 - accuracy: 1.0000 - val_loss: 7.4961e-06 - val_accuracy: 1.0000
Epoch 6/25
17/17 [==============================] - 8s 440ms/step - loss: 1.4716e-04 - accuracy: 1.0000 - val_loss: 8.7298e-06 - val_accuracy: 1.0000
Epoch 7/25
17/17 [==============================] - 8s 447ms/step - loss: 1.1897e-04 - accuracy: 1.0000 - val_l

In [24]:
import cv2
import numpy as np

mood = ['Angry', 'Confused', 'Happy', 'Sad']
cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
camera = cv2.VideoCapture(0)

while True:
    ret, frame = camera.read()

    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    roi = cascade.detectMultiScale(gray, 1.5, 5)

    for (x,y,w,h) in roi:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (224,224))
        face = np.expand_dims(face, axis=0)

        pred = model.predict(face)
        label = np.argmax(pred)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, mood[label], (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 23ms/step
